## 模型融合概述

本文转自：一份完备的集成学习手册[https://mp.weixin.qq.com/s/Pkc8KyDZ53ZGO5lNLVjoBg]

根据该文章稍作修改，文档中具体算法的实现参见本项目其他文档。

### 目录
#### 1. 基本的融合技术

    1.1 最大化

    1.2 平均化

    1.3 加权平均

#### 2. 高级融合技术

    2.1 Stacking

    2.2 Blending

### 1.  基本的集成技术（略）

### 2. 高级集成技术

#### 2.1 Stacking

Stacking 是使用多个模型（例如决策树、KNN、SVM）来构建新的模型的集成技术。该模型在测试集上进行预测。下面是一个简单的 Stacking 集成的详细步骤解释。

1）将训练集划分为 10 个子集。

2）在其中 9 个子集上训练一个基本模型（例如决策树模型），在第 10 个子集上进行测试。遍历每个子集，重复进行 10 次。得到的 DT 长度与 Train set 相同。
![1](https://mmbiz.qpic.cn/mmbiz_png/hflWRBRSEZ6Ccb2zNuL3r3TvQRPIFykKfHaYfuvsTku91N6xFzA7iaMF8AjA0y9GTXPZicenOYycBubETglibhZkA/640?tp=webp&wxfrom=5&wx_lazy=1&wx_co=1)
3）在整个训练集上使用该模型（决策树）进行建模。

4）使用建模的模型在测试集上进行测试。
![2](https://mmbiz.qpic.cn/mmbiz_png/hflWRBRSEZ6Ccb2zNuL3r3TvQRPIFykKJ0nc1yYs53yEcj0eZoVKfA65XFPoCkyXZdxVTtoLaElLy5JiaGGqfFw/640?tp=webp&wxfrom=5&wx_lazy=1&wx_co=1)

5）使用另一种算法（例如 knn），重复步骤 2～4，作用在 Train set 和 Test set 上，得到另一组值。
![3](https://mmbiz.qpic.cn/mmbiz_png/hflWRBRSEZ6Ccb2zNuL3r3TvQRPIFykKgOkpy85tORSHowuIn0hQ88SniclGEG7y4N1C56S9XcAVvhjsqx9mVQA/640?tp=webp&wxfrom=5&wx_lazy=1&wx_co=1)

6）使用得到的 DT 和 knn 组合作为新的特征 TRAIN PREDICTION SET，训练新的模型（例如逻辑回归）。
![4](https://mmbiz.qpic.cn/mmbiz_png/hflWRBRSEZ6Ccb2zNuL3r3TvQRPIFykKY3jsZa3SSys3K3LiaxibG1e38ap4B1cG8FswicPACHrfaRuJibveu02w4g/640?tp=webp&wxfrom=5&wx_lazy=1&wx_co=1)

7）使用训练好的模型对 TEST PREDICTION SET 进行预测。

In [5]:
# 示例
# 为了将问题简单化，所创建的 Stacking 模型只有两层。第一层是建立决策树和 knn 模型，两个基学习器，第二层是建立逻辑回归模型。实际应用中可以使用多个层次的复杂结构。
# 首先，我们需要定义一个函数对 n 折训练集和测试集进行预测，该函数返回每个模型对训练集和测试集的预测结果。

import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn import tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

# 使用Kaggle Titanic数据集
train = pd.read_csv('../../Data/train_fixed.csv')
test = pd.read_csv('../../Data/test_fixed.csv')
train_X = train.drop(['Survived'], axis=1)
train_y = train['Survived']

# for train, test in folds.split(train_X, train_y):
#     print('train: ', train)
#     print('test: ', test)
#     print('----------------------------------')

def Stacking(model, train_X, train_y, test_X, n_fold, model_name):
    folds=StratifiedKFold(n_splits=n_fold, random_state=1)
    test_pred_nfolds = pd.DataFrame(np.empty((test_X.shape[0], n_fold)), dtype=float, columns=['test_pred_fold' + str(i) for i in np.arange(0,n_fold)])
    train_pred = pd.DataFrame(np.empty((train_X.shape[0], 1)), dtype=float, columns=[model_name])
    fold = 0
    for train_index, val_index in folds.split(train_X, train_y.values):
        x_train, x_val = train_X.iloc[train_index], train_X.iloc[val_index]
        y_train, y_val = train_y.iloc[train_index], train_y.iloc[val_index]

        # 训练模型
        model.fit(X=x_train, y=y_train)

        # 判断该折验证集模型效果
        # y_val_hat = model.predict(x_val) & y_val

        # 合并各折预测值，得到训练集整体预测值；每折都对测试集进行一次预测，最后取平均作为测试集最终取值 
        train_pred[model_name].iloc[val_index] = model.predict(x_val)
        test_pred_nfolds.iloc[:, fold] = model.predict(test_X)
        
        fold += 1
    
    test_pred = test_pred_nfolds.mean(axis=1)

    return train_pred, test_pred

# 构建两个基本模型：决策树和 knn
model1 = tree.DecisionTreeClassifier(random_state=1)
train_pred1, test_pred1 =Stacking(model=model1, train_X=train_X, train_y=train_y, test_X=test, n_fold=10, model_name='tree')

model2 = KNeighborsClassifier()
train_pred2, test_pred2 =Stacking(model=model2, train_X=train_X, train_y=train_y, test_X=test, n_fold=10, model_name='knn')

# 使用逻辑回归，进行训练和预测。
df = pd.concat([train_pred1, train_pred2], axis=1)
df_test = pd.concat([test_pred1, test_pred2], axis=1)
df_test.columns = ['tree', 'knn']

model = LogisticRegression(random_state=1)
model.fit(df, train_y)
pred_y = model.predict(df_test)

#### 2.2 Blending

Blending 与 Stacking 类似，但是仅从训练集上划分一部分作为 holdout（验证集），没有使用 k 折验证。Holdout 集结果作为下一层的训练数据。下面是 Blending 的详细步骤解释。

    1. 将所有的训练数据划分为训练集和验证集。
![blending1](https://mmbiz.qpic.cn/mmbiz_png/hflWRBRSEZ6Ccb2zNuL3r3TvQRPIFykKZAp6tMLxbwkcCAf31qAYzDGGRq6RrErF81zkDB8CQqXBCAQcaxjByw/640?tp=webp&wxfrom=5&wx_lazy=1&wx_co=1)
    
    2. 在训练集上训练模型。

    3. 在验证集和整体测试集上进行模型测试。

    4. 验证集和测试结果作为元特征，进行第二层的模型训练。

    5. 使用该模型在整体测试集的元特征上进行模型验证。

In [85]:
# 示例

import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn import tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

# 使用Kaggle Titanic数据集
train_dt = pd.read_csv('../../Data/train_fixed.csv')
test_dt = pd.read_csv('../../Data/test_fixed.csv')

train_index = pd.Series(np.arange(train.shape[0])).sample(n=int(train.shape[0]/4*3))
val_index = pd.Series([x for x in np.arange(train.shape[0]) if x not in train_index])

train = train_dt.iloc[train_index]
val = train_dt.iloc[val_index]
test_X = test_dt

train_X = train.drop(['Survived'], axis=1)
train_y = train['Survived']
val_X = val.drop(['Survived'], axis=1)
val_y = val['Survived']

# 首先，我们在训练集上训练两个模型：决策树和 knn，以便在验证集上作出预测。
model1 = tree.DecisionTreeClassifier()
model1.fit(train_X, train_y)
val_pred1 = model1.predict(val_X)
test_pred1 = model1.predict(test_X)
val_pred1 = pd.DataFrame(val_pred1, index=val_X.index, columns=['tree'])
test_pred1 = pd.DataFrame(test_pred1, columns=['tree'])

model2 = KNeighborsClassifier()
model2.fit(train_X, train_y)
val_pred2 = model2.predict(val_X)
test_pred2 = model2.predict(test_X)
val_pred2 = pd.DataFrame(val_pred2, index=val_X.index, columns=['knn'])
test_pred2 = pd.DataFrame(test_pred2, columns=['knn'])

# 然后，结合验证集的元特征，训练逻辑回归模型，在测试集上进行验证。
df_val_layer2 = pd.concat([val_X, val_pred1, val_pred2],axis=1)
df_test_layer2 = pd.concat([test_X, test_pred1, test_pred2],axis=1)

model_layer2 = LogisticRegression()
model_layer2.fit(df_val_layer2, val_y)
test_y_pred = model_layer2.predict(df_test_layer2)
# roc_auc_score(test_y, test_y_pred)